In [3]:
import numpy as np
import pandas as pd
import wbgapi as wb
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [4]:
# =========================================================
# 1. OWID – Renewable production (TWh)
# =========================================================

df_renew = pd.read_csv(
    "https://ourworldindata.org/grapher/modern-renewable-prod.csv?v=1&csvType=full&useColumnShortNames=true",
    storage_options={"User-Agent": "OWID fetch"}
)

# detect data column
val_cols = [c for c in df_renew.columns if c not in ["Entity","Code","Year"]]
val = val_cols[0]

df_renew = df_renew.rename(columns={
    "Code":"iso3",
    "Year":"year",
    val:"renewable_prod_TWh"
})[["iso3","year","renewable_prod_TWh"]]

df_renew = df_renew.dropna(subset=["iso3"])

KeyError: "['iso3'] not in index"

In [27]:
# =========================================================
# 2.5 Fetch World Bank Indicators - Positional Fix
# =========================================================

# 1. Fetch data (mrv=10 usually returns economy as index)
df_wb = wb.data.DataFrame('NY.GDP.PCAP.CD', countries, mrv=10)

# 2. Reset the index so 'economy' and 'time' become actual columns
df_wb = df_wb.reset_index()

# 3. Rename columns by POSITION to guarantee success
# We assume: Column 0 is the Country (iso3), Column 1 is Year/Time, Column 2 is Value
# Check how many columns we have to decide if it's "Long" or "Wide" format
if len(df_wb.columns) >= 3 and df_wb.shape[1] < 10:
    # This is LONG format: [Country, Year, Value]
    df_wb.columns = ['iso3', 'year', 'gdp_per_capita'] + list(df_wb.columns[3:])
else:
    # This is WIDE format: [Country, YR2010, YR2011...]
    # Rename the first column to iso3, then melt the rest
    new_cols = list(df_wb.columns)
    new_cols[0] = 'iso3'
    df_wb.columns = new_cols

    df_wb = df_wb.melt(id_vars=['iso3'], var_name='year', value_name='gdp_per_capita')

# 4. Final Clean
df_wb['year'] = df_wb['year'].astype(str).str.replace('YR', '').astype(int)

print("Success! Columns:", df_wb.columns.tolist())
print(df_wb.head(3))

Success! Columns: ['iso3', 'year', 'gdp_per_capita']
  iso3  year  gdp_per_capita
0  ABW  2015    27458.220154
1  AFG  2015      565.569730
2  AGO  2015     3641.728939


In [28]:
# =========================================================
# 3. OWID – CO₂ emissions (Refined)
# =========================================================

co2 = pd.read_csv("https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv")
co2 = co2[["iso_code","year","co2"]].rename(columns={
    "iso_code":"iso3",
    "co2":"co2_tonnes"
}).dropna(subset=["iso3"]) # Clean up regions like 'OWID_WRL'

In [29]:
# =========================================================
# 4. Merge all data into df
# =========================================================

df = (
    df_wb
    .merge(df_renew, on=["iso3","year"], how="left")
    .merge(co2, on=["iso3", "year"], how="left")
)

# Merge with meta to get country names back
df = df.merge(meta, on="iso3", how="left")

print(f"Merge complete. Shape: {df.shape}")

Merge complete. Shape: (2170, 6)


In [31]:
# =========================================================
# 5. Construct variables (Corrected)
# =========================================================

# Handle NaNs in renewables before the boolean check if necessary
# (NaN > 0 is False, which becomes 0. If you want NaNs to stay NaNs, handle separately)
df["has_renewables"] = (df["renewable_prod_TWh"].fillna(0) > 0).astype(int)

df["renewable_prod_log"] = np.log1p(df["renewable_prod_TWh"])

# FIXED: Use 'gdp_per_capita' instead of 'GDP_Current_USD'
df["gdp_log"] = np.log(df["gdp_per_capita"].replace(0, np.nan))

df["co2_log"] = np.log1p(df["co2_tonnes"])

df = df.replace([np.inf, -np.inf], np.nan)

print("Variables constructed successfully.")
print(df[["iso3", "year", "gdp_log", "co2_log"]].head())

Variables constructed successfully.
  iso3  year    gdp_log   co2_log
0  ABW  2015  10.220421  0.640801
1  AFG  2015   6.337834  2.340266
2  AGO  2015   8.200214  3.350080
3  ALB  2015   8.342730  1.742569
4  AND  2015  10.562430  0.381855


In [ ]:
# =========================================================
# 6. Estimate U-curves
# =========================================================

# --- Production ---
df_prod = df[df["has_renewables"]==1].dropna(
    subset=["renewable_prod_log","gdp_log"]
).copy()
df_prod["gdp_log_sq"] = df_prod["gdp_log"]**2

prod = sm.OLS(df_prod["renewable_prod_log"],
              sm.add_constant(df_prod[["gdp_log","gdp_log_sq"]])).fit()
tp_prod = -prod.params["gdp_log"]/(2*prod.params["gdp_log_sq"])

# --- Share ---
df_share = df.dropna(
    subset=["Renewable_Energy_Share_Pct","gdp_log"]
).copy()
df_share["gdp_log_sq"] = df_share["gdp_log"]**2

share = sm.OLS(df_share["Renewable_Energy_Share_Pct"],
               sm.add_constant(df_share[["gdp_log","gdp_log_sq"]])).fit()
tp_share = -share.params["gdp_log"]/(2*share.params["gdp_log_sq"])

# --- CO₂ ---
df_co2 = df.dropna(
    subset=["co2_log","gdp_log"]
).copy()
df_co2["gdp_log_sq"] = df_co2["gdp_log"]**2

co2m = sm.OLS(df_co2["co2_log"],
              sm.add_constant(df_co2[["gdp_log","gdp_log_sq"]])).fit()
tp_co2 = -co2m.params["gdp_log"]/(2*co2m.params["gdp_log_sq"])

In [ ]:
# =========================================================
# 7. Plot & save figures
# =========================================================

x = np.linspace(df["gdp_log"].min(), df["gdp_log"].max(), 300)

# Global plot style for posters
plt.rcParams.update({
    "font.size": 11,
    "axes.titlesize": 13,
    "axes.labelsize": 11,
    "figure.dpi": 300
})


# --- Production ---
plt.figure(figsize=(7,5))

sc = plt.scatter(
    df_prod["gdp_log"],
    df_prod["renewable_prod_log"],
    c=df_prod["gdp_log"],      # color by income
    cmap="magma",
    alpha=0.5,
    s=18
)

plt.plot(
    x,
    prod.params["const"]
    + prod.params["gdp_log"]*x
    + prod.params["gdp_log_sq"]*x**2,
    color="black",
    linewidth=2
)

plt.axvline(tp_prod, linestyle="--", color="black", alpha=0.7)
plt.colorbar(sc, label="Log GDP")

plt.xlabel("Log GDP (current USD)")
plt.ylabel("Log renewable energy production")
plt.title("Renewable production vs economic growth")

plt.tight_layout()
plt.savefig("ucurve_production.png")
plt.close()


# --- Share ---
plt.figure(figsize=(7,5))

sc = plt.scatter(
    df_share["gdp_log"],
    df_share["Renewable_Energy_Share_Pct"],
    c=df_share["gdp_log"],
    cmap="magma",
    alpha=0.5,
    s=18
)

plt.plot(
    x,
    share.params["const"]
    + share.params["gdp_log"]*x
    + share.params["gdp_log_sq"]*x**2,
    color="black",
    linewidth=2
)

plt.axvline(tp_share, linestyle="--", color="black", alpha=0.7)
plt.colorbar(sc, label="Log GDP")

plt.xlabel("Log GDP (current USD)")
plt.ylabel("Renewable energy share (%)")
plt.title("Renewable share vs economic growth")

plt.tight_layout()
plt.savefig("ucurve_share.png")
plt.close()


# --- CO₂ ---
plt.figure(figsize=(7,5))

sc = plt.scatter(
    df_co2["gdp_log"],
    df_co2["co2_log"],
    c=df_co2["gdp_log"],
    cmap="magma",
    alpha=0.5,
    s=18
)

plt.plot(
    x,
    co2m.params["const"]
    + co2m.params["gdp_log"]*x
    + co2m.params["gdp_log_sq"]*x**2,
    color="black",
    linewidth=2
)

plt.axvline(tp_co2, linestyle="--", color="black", alpha=0.7)
plt.colorbar(sc, label="Log GDP")

plt.xlabel("Log GDP (current USD)")
plt.ylabel("Log CO₂ emissions")
plt.title("CO₂ emissions vs economic growth")

plt.tight_layout()
plt.savefig("ucurve_co2.png")
plt.close()

In [1]:
# =========================================================
# 8. Report turning points
# =========================================================

print("GDP turning point – Renewable production:", round(np.exp(tp_prod)))
print("GDP turning point – Renewable share:", round(np.exp(tp_share)))
print("GDP turning point – CO₂ emissions:", round(np.exp(tp_co2)))


KeyError: 'id'